# <font color=yellow> **Modelo sísmico**

In [51]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import resample
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## <font color=yellow> **Criando o modelo sísmico**


In [128]:
# depth axis
z = np.arange(0, 1000, 1)
n_z = len(z)

# time axis
t = np.arange(0, 4.0, 0.004)
n_t = len(t)

# CMP
cmp = np.arange(1, 11)
n_cmp = len(cmp)

# reservoir thikness
reservoir = 550 # meters

# vp values for each layer
vp_c1 = 2.8 # sandstone
vp_c2 = 2.4
vp_c3 = 3.2 # high porosity sandstone (reservoir)
vp_c4 = 2.6
vp_c5 = 5.5 # limestone

# vs values for each layer
vs_c1 = 1.5 # sandstone
vs_c2 = 1.1
vs_c3 = 1.7 # high porosity sandstone (reservoir)
vs_c4 = 1.2
vs_c5 = 3.0 # limestone

# rho values for each layer
rho_c1 = 2.3 # sandstone
rho_c2 = 2.4
rho_c3 = 2.0 # high porosity sandstone (reservoir)
rho_c4 = 2.3
rho_c5 = 2.7 # limestone

# vp model
vp = np.zeros((n_z, n_cmp))
vp[0:250, :] = vp_c1
vp[250:500, :] = vp_c2
vp[500:reservoir, :] = vp_c3
vp[reservoir:750, :] = vp_c4
vp[750:1000, :] = vp_c5

for i in range(n_cmp):
    vp[0:250, i] = vp_c1 + (i/1000)
    vp[250:500, i] = vp_c2 + 5*(i/1000)
    vp[500:reservoir, i] = vp_c3 + 10*(i/1000)
    vp[reservoir:750, i] = vp_c4 + (i/1000)
    vp[750:1000, i] = vp_c5 + 10*(i/1000)

# vs model
vs = np.zeros((n_z, n_cmp))
vs[0:100, :] = vs_c1
vs[250:500, :] = vs_c2
vs[500:reservoir, :] = vs_c3
vs[reservoir:750, :] = vs_c4
vs[750:1000, :] = vs_c5

for i in range(n_cmp):
    vs[0:100, i] = vs_c1 + (i/1000)
    vs[250:500, i] = vs_c2 + 5*(i/1000)
    vs[500:reservoir, i] = vs_c3 + 10*(i/1000)
    vs[reservoir:750, i] = vs_c4 + (i/1000)
    vs[750:1000, i] = vs_c5 + 10*(i/1000)

# rho model
rho = np.zeros((n_z, n_cmp))
rho[0:250, :] = rho_c1
rho[250:500, :] = rho_c2
rho[500:reservoir, :] = rho_c3
rho[reservoir:750, :] = rho_c4
rho[750:1000, :] = rho_c5

for i in range(n_cmp):
    rho[0:250, i] = rho_c1 + (i/1000)
    rho[250:500, i] = rho_c2 + 5*(i/1000)
    rho[500:reservoir, i] = rho_c3 + 10*(i/1000)
    rho[reservoir:750, i] = rho_c4 + (i/1000)
    rho[750:1000, i] = rho_c5 + 10*(i/1000)

In [129]:
# vp model plot
fig = go.Figure()

fig.add_trace(go.Heatmap(z=vp[::-1], zmin=vp.min(), zmax=vp.max(), colorscale='Jet',
                         x=[str(c) for c in cmp], y=[str(int(d)) for d in z[::-1]]))

fig.update_layout(
    title="Compressional Velocity (m/s)",
    title_x=0.5,
    xaxis=dict(title="CMP"),
    yaxis=dict(title="Depth (m)"),
    coloraxis=dict(colorbar=dict(title="Compressional velocity (m/s)", len=0.5, y=0.5)),
    height=500, width=500,
    font=dict(size=18)
)

fig.update_yaxes(tickvals=list(range(0, 1001, 100)))

fig.show()

In [130]:
# vs model plot
fig = go.Figure()

fig.add_trace(go.Heatmap(z=vs[::-1], zmin=vs.min(), zmax=vs.max(), colorscale='Jet',
                         x=[str(c) for c in cmp], y=[str(int(d)) for d in z[::-1]]))

fig.update_layout(
    title="Shear velocity (m/s)",
    title_x=0.5,
    xaxis=dict(title="CMP"),
    yaxis=dict(title="Depth (m)"),
    coloraxis=dict(colorbar=dict(title="Shear velocity (m/s)", len=0.5, y=0.5)),
    height=500, width=500,
    font=dict(size=18)
)

fig.update_yaxes(tickvals=list(range(0, 1001, 100)))

fig.show()

In [110]:
# rho model plot
fig = go.Figure()

fig.add_trace(go.Heatmap(z=rho[::-1], zmin=rho.min(), zmax=rho.max(), colorscale='Jet',
                         x=[str(c) for c in cmp], y=[str(int(d)) for d in z[::-1]]))

fig.update_layout(
    title="Density (kg/m³)",
    title_x=0.5,
    xaxis=dict(title="CMP"),
    yaxis=dict(title="Depth (m)"),
    coloraxis=dict(colorbar=dict(title="Velocity (m/s)", len=0.5, y=0.5)),
    height=500, width=500,
    font=dict(size=18)
)

fig.update_yaxes(tickvals=list(range(0, 901, 100)))

fig.show()

In [131]:
# vp and rho plot
fig = make_subplots(rows=1, cols=2, subplot_titles=('Velocity', 'Density'))

fig.add_trace(go.Scatter(x=vp[:,0], mode='lines', name='Seismic trace',
                         line=dict(color='darkblue')), row=1, col=1)
fig.add_trace(go.Scatter(x=rho[:,0], mode='lines', name='Seismic trace',
                         line=dict(color='crimson')), row=1, col=2)

# xaxis properties
fig.update_xaxes(title_text='(m/s)', row=1, col=1,
                 title_font=dict(size=18), tickfont=dict(size=16))
fig.update_xaxes(title_text='(kg/m³)', row=1, col=2,
                 title_font=dict(size=18), tickfont=dict(size=16))

# yaxis properties
fig.update_yaxes(title_text='Depth (m)', row=1, col=1, autorange="reversed",
                 title_font=dict(size=18), tickfont=dict(size=16))
fig.update_yaxes(row=1, col=2, autorange="reversed",
                 title_font=dict(size=18), tickfont=dict(size=16))

fig.update_layout(height=500, width=650, showlegend=False, title_text='Seismic trace and wavelet from 2D model',
                  title_font=dict(size=20), title_x=0.5)

fig.update_annotations(font=dict(size=18))

fig.show()

## <font color=yellow> **Using Gasssmann to create the original trace**



In [132]:
# selecting CMP
vp_trace = vp[:,0]
vs_trace = vs[:,0]
rho_trace = rho[:,0]

# selecting the reservoir layer
Vp = vp_trace[500:reservoir]   # vp (km/s)
Vs = vs_trace[500:reservoir]   # vs (km/s)
Rho = rho_trace[500:reservoir] # rho (g/cm3)

# initial reservoir conditions
Por = 0.30     # Porosity (%)
K_min = 36     # rock bulk modulus (GPa) (GPa)
Rho_min = 2.65 # rock density (g/cm3)

# initial reservoir fluids conditions
Swe = 0          # no invasion (filtrate saturation)
K_hc = 1.15      # oil bulk modulus (GPa)
K_filtrate = 3.0 # filtrate bulk modulus (GPa)
rho_hc = 0.8     # oil density (g/cm3)
rho_mud = 1.3    # filtrate density (g/cm3)

# shear modulus
mu = Rho * Vs**2

# dry rock bulk modulus
K_dry = K_min * (1 - Por)

# fluid bulk modulus
K_fluid = 1 / ((Swe / K_filtrate) + ((1 - Swe) / K_hc))

# fluid density
Rho_fluid = (1 - Swe) * rho_hc + Swe * rho_mud

# saturated bulk modulus
K_sat = K_dry +( ((1-(K_dry/K_min))**2)/(Por/K_fluid + (1-Por)/K_min + K_dry/K_min**2))

# new density
Rho_new = Rho_min + Por * Rho_fluid

# new vs
Vs_new = np.sqrt(mu / Rho_new)

# new vp
Vp_new = np.sqrt((K_sat + ((4 / 3) * mu)) / Rho_new)

# Updating the results for each saturation
vp_values = Vp_new  # Corrigido para atribuir valor a cada elemento
vs_values = Vs_new  # Corrigido para atribuir valor a cada elemento
rho_values = Rho_new  # Corrigido para atribuir valor a cada elemento

# New compressional velocity
vp_trace[0:250] = vp_c1
vp_trace[250:500] = vp_c2
vp_trace[500:reservoir] = vp_values
vp_trace[reservoir:750] = vp_c4
vp_trace[750:1000] = vp_c5

# New shear velocity
vs_trace[0:250] = vs_c1
vs_trace[250:500] = vs_c2
vs_trace[500:reservoir] = vs_values
vs_trace[reservoir:750] = vs_c4
vs_trace[750:1000] = vs_c5

# New densidade
rho_trace[0:250] = rho_c1
rho_trace[250:500] = rho_c2
rho_trace[500:reservoir] = rho_values
rho_trace[reservoir:750] = rho_c4
rho_trace[750:1000] = rho_c5

In [133]:
# vp and rho plot
fig = make_subplots(rows=1, cols=2, subplot_titles=('Velocity', 'Density'))

fig.add_trace(go.Scatter(x=vp_trace, mode='lines', name='Seismic trace',
                         line=dict(color='darkblue')), row=1, col=1)
fig.add_trace(go.Scatter(x=rho_trace, mode='lines', name='Seismic trace',
                         line=dict(color='crimson')), row=1, col=2)

# xaxis properties
fig.update_xaxes(title_text='(m/s)', row=1, col=1,
                 title_font=dict(size=18), tickfont=dict(size=16))
fig.update_xaxes(title_text='(kg/m³)', row=1, col=2,
                 title_font=dict(size=18), tickfont=dict(size=16))

# yaxis properties
fig.update_yaxes(title_text='Depth (m)', row=1, col=1, autorange="reversed",
                 title_font=dict(size=18), tickfont=dict(size=16))
fig.update_yaxes(row=1, col=2, autorange="reversed",
                 title_font=dict(size=18), tickfont=dict(size=16))

fig.update_layout(height=500, width=650, showlegend=False, title_text='Seismic trace and wavelet from 2D model',
                  title_font=dict(size=20), title_x=0.5)

fig.update_annotations(font=dict(size=18))

fig.show()

## <font color=yellow> **Generating the seismic trace**

In [135]:
# wavelet Ricker
f = 20  # Hz
w = np.zeros(len(t))
for i in range(len(t)):
    w[i] = (1 - (2 * np.pi ** 2 * f ** 2 * (t[i] - 0.08) ** 2)) * np.exp(-np.pi ** 2 * f ** 2 * (t[i] - 0.08) ** 2)

In [136]:
# impedance
ip = np.zeros((n_z))
ip = vp_trace * rho_trace

# reflectivity
R = np.zeros((n_z))
for i in range(n_z - 1):
  R[i] = (ip[i + 1] - ip[i]) / (ip[i + 1] + ip[i])

# Convolution
synt = np.zeros(n_z)
synt = np.convolve(R, w)

# creating a seismic trace with noise
synt_r = np.zeros(n_z)
for i in range(n_z):
    h = np.random.rand(1) / 25
    synt_r[i] = synt[i] + h

<ipython-input-136-cfff637ebb08>:18: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



In [137]:
# synthetic seismic trace plot
fig = make_subplots(rows=1, cols=4, subplot_titles=('Impedance', 'Reflectivity', 'Wavelet', 'Seismic trace'))

fig.add_trace(go.Scatter(x=ip, y=t, name='Impedance',
                         line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=R, y=t, name='Reflectivity',
                         line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=w, y=t, name='Wavelet',
                         line=dict(color='blue')), row=1, col=3)
fig.add_trace(go.Scatter(x=synt_r, y=t, name='Seismic trace',
                         line=dict(color='blue')), row=1, col=4)

# xaxis properties
fig.update_xaxes(title_text='kg/(m²s)', row=1, col=1,
                 title_font=dict(size=18), tickfont=dict(size=16))
fig.update_xaxes(title_text='Unitless', row=1, col=2,
                 title_font=dict(size=18), tickfont=dict(size=16))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=3,
                 title_font=dict(size=18), tickfont=dict(size=16))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=4,
                 title_font=dict(size=18), tickfont=dict(size=16))

# yaxis properties
fig.update_yaxes(title_text="Time (s)", row=1, col=1, autorange="reversed",
                 title_font=dict(size=18), tickfont=dict(size=16))
fig.update_yaxes(autorange="reversed", row=1, col=2,
                 title_font=dict(size=18), tickfont=dict(size=16))
fig.update_yaxes(autorange="reversed", range=[0, 3], row=1, col=3,
                 title_font=dict(size=18), tickfont=dict(size=16))
fig.update_yaxes(autorange="reversed", row=1, col=4,
                 title_font=dict(size=18), tickfont=dict(size=16))

fig.update_layout(height=600, width=1200, showlegend=False,
                  title_text=f'Seismic model: thickness: {len(vp_values)}m, porosity: {round(Por*100)}%, frequency: {f} Hz ',
                  title_font=dict(size=20), title_x=0.5)

fig.update_annotations(font=dict(size=18))

fig.show()

In [138]:
# saving the seismic trace and wavelet
np.savetxt(f'/content/drive/MyDrive/doutorado/dados/trace_{len(vp_values)}_{round(Por*100)}_{f}.txt', synt_r)
np.savetxt(f'/content/drive/MyDrive/doutorado/dados/wav_{f}.txt', w)